In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
number = re.compile('[\d,]+')
def get_first_number(val):
    matched = number.match(str(val))
    if matched:
        return int(matched.group())
    else:
        return np.nan

satisfaction_strs = {
    'I love my job': 10,
    'I\'m somewhat satisfied with my job': 7.5,
    'I\'m neither satisfied nor dissatisfied with my job': 5,
    'I\'m somewhat dissatisfied with my job': 2.5,
    'I hate my job': 0,
}

binary_labels = [
    'Lang & Tech',
    'Training & Education',
    'How can companies improve interview process',
    'Why try Stack Overflow Careers',
    'Most important aspect of new job opportunity',
    'Most annoying about job search',
    'Appealing message traits',
    'Most urgent info about job opportunity',
    'Who do you want to communicate with about a new job opportunity',
    'Why use Stack Overflow',
    'Why answer',
    'Source control used',
]

numeric_labels = [
    'Age',
    'Years IT / Programming Experience',
    'Compensation: midpoint'
]

yes_no_labels = [
    'Changed Jobs in last 12 Months'
]


In [ ]:
# 2015 preproc
data = pd.read_csv('data/2015.csv')
to_drop = [label for label in data if 'write-in' in label.lower()]

data['Job Satisfaction'] = data['Job Satisfaction']\
                                    .map(satisfaction_strs).astype('float')
data = data[data['Job Satisfaction'].notnull()]

to_drop.append('Compensation')

data['Age'] = data['Age'].map(get_first_number).astype('float')
data['gender_M'] = (data['Gender'] == 'Male').astype('int8')
data['gender_F'] = (data['Gender'] == 'Female').astype('int8')
to_drop.append('Gender')
to_drop.append('Prefered Source Control')

bin_labels = [key for key in data if any(label in key for label in binary_labels)]
data[bin_labels] = data[bin_labels].apply(lambda col: col.notnull().astype('int8'))

data[numeric_labels] = data[numeric_labels].applymap(get_first_number)
data[yes_no_labels] = data[yes_no_labels]\
                                .apply(lambda col: col.map({'Yes': 1, 'No': 0}))\
                                .fillna(0)

data.drop(to_drop, axis=1, inplace=True)
data = pd.get_dummies(data)

print 'Columns with NaNs:'
for key in data:
    if data[key].isnull().any():
        print key
data.fillna(data.mean(), inplace=True)
print 'Filling with mean of column'